# MinIO Test

In [16]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("spark://spark:7077")
         .appName("MinIO-Integration")
         # Add postgres jar
         .config('spark.jars.packages', 'com.amazonaws:aws-java-sdk-bundle:1.11.315,org.apache.hadoop:hadoop-aws:3.2.0') 
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "minio")
         .config("spark.hadoop.fs.s3a.secret.key", "miniosecret")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         #.config("fs.s3a.multipart.size", "104857600")
         #set('fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider')
         #set('fs.s3a.session.token', credentials['SessionToken'])
         .getOrCreate())

In [17]:
train = spark.read\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .csv("s3a://stage/train.csv")
train.registerTempTable("train")

In [18]:
train.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Loan Amount: integer (nullable = true)
 |-- Funded Amount: integer (nullable = true)
 |-- Funded Amount Investor: double (nullable = true)
 |-- Term: integer (nullable = true)
 |-- Batch Enrolled: string (nullable = true)
 |-- Interest Rate: double (nullable = true)
 |-- Grade: string (nullable = true)
 |-- Sub Grade: string (nullable = true)
 |-- Employment Duration: string (nullable = true)
 |-- Home Ownership: double (nullable = true)
 |-- Verification Status: string (nullable = true)
 |-- Payment Plan: string (nullable = true)
 |-- Loan Title: string (nullable = true)
 |-- Debit to Income: double (nullable = true)
 |-- Delinquency - two years: integer (nullable = true)
 |-- Inquires - six months: integer (nullable = true)
 |-- Open Account: integer (nullable = true)
 |-- Public Record: integer (nullable = true)
 |-- Revolving Balance: integer (nullable = true)
 |-- Revolving Utilities: double (nullable = true)
 |-- Total Accounts: intege

In [19]:
train.groupBy("Application Type").sum("Loan Amount").show()

+----------------+----------------+
|Application Type|sum(Loan Amount)|
+----------------+----------------+
|           JOINT|         2162735|
|      INDIVIDUAL|      1134514793|
+----------------+----------------+



In [20]:
train.registerTempTable("train")

In [21]:
App_Types = spark.sql("""
    SELECT `Application Type` as Application_Type, sum(`Loan Amount`) as Balance
    FROM train
    group by `Application Type`
""")

In [22]:
App_Types.show()

+----------------+----------+
|Application_Type|   Balance|
+----------------+----------+
|           JOINT|   2162735|
|      INDIVIDUAL|1134514793|
+----------------+----------+



In [9]:
App_Types.write.parquet("s3a://stage/app_types")

In [23]:
test = spark.read.parquet("s3a://stage/app_types")

In [24]:
test.printSchema()

root
 |-- Application_Type: string (nullable = true)
 |-- Balance: long (nullable = true)



In [25]:
test.show()

+----------------+----------+
|Application_Type|   Balance|
+----------------+----------+
|      INDIVIDUAL|1134514793|
|           JOINT|   2162735|
+----------------+----------+



In [26]:
spark.stop()

21/12/27 22:35:24 WARN Dispatcher: Message RemoteProcessDisconnected(172.21.0.3:59350) dropped. Could not find BlockManagerMasterHeartbeat.
